In [1]:
import open3d as o3d
import numpy as np
import colour
from pint import UnitRegistry
import matplotlib.pyplot as plt

In [ ]:
pcd = o3d.io.read_point_cloud("test_scan_100000.pcd")

print(pcd)
print(np.asarray(pcd.points))
#o3d.visualization.draw_geometries([pcd])

In [ ]:
diameter = np.linalg.norm(np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound()))
o3d.visualization.draw_geometries([pcd])

In [ ]:
pcd = o3d.io.read_point_cloud("RUGeoMuseum_2.pts")
print(pcd)
print(np.asarray(pcd.points))

In [ ]:
min_bound = np.asarray(pcd.get_max_bound())
min_bound

In [ ]:
max_bound = np.asarray(pcd.get_min_bound())
max_bound

In [ ]:
mid_position = (min_bound + max_bound) / 2
mid_position

In [ ]:
camera = [6.317, -2.375, -57.286]
radius = 5

print("Get all points that are visible from given view point")
_, pt_map = pcd.hidden_point_removal(camera, radius)

print("Visualize result")
pcd2 = pcd.select_by_index(pt_map)
o3d.visualization.draw_geometries([pcd2])

# Register Point Cloud from Virtual Scans

In [ ]:
import os

In [ ]:
dir_name = "test_scan/"

scan_config = (0, 3.5, 100)

scan_loc = np.linspace(scan_config[0], scan_config[1], scan_config[2])
    
Pcd = o3d.geometry.PointCloud()

n = 0
for i in range(scan_config[2]):
    temp = o3d.io.read_point_cloud("test_scan/test_scan_%d00000.pcd" % (i))
    temp.translate((0,scan_loc[n],0))
    Pcd += temp
    n += 1
    
print(np.asarray(Pcd))

In [ ]:
o3d.visualization.draw_geometries([Pcd])

# UVC Visibility Analysis using Mesh-Generated Point Cloud

In [2]:
mesh = o3d.io.read_triangle_mesh("bedroom.stl")
mesh.compute_vertex_normals()
print(mesh)

geometry::TriangleMesh with 198489 points and 66163 triangles.


In [3]:
mesh_box = o3d.geometry.TriangleMesh.create_box(width=1.0,
                                                height=1.0,
                                                depth=1.0)
mesh_box.paint_uniform_color([0.9, 0.1, 0.1])
mesh_box.compute_vertex_normals()
mesh_box.translate([-10, 0, 0])

geometry::TriangleMesh with 8 points and 12 triangles.

In [4]:
o3d.visualization.draw_geometries(
        [mesh_box, mesh])

In [5]:
mesh_comb = mesh + mesh_box

In [6]:
o3d.visualization.draw_geometries([mesh_comb])

In [ ]:
Total_points = 500000

pcd = mesh_comb.sample_points_poisson_disk(Total_points)
diameter = np.linalg.norm(np.asarray(pcd.get_max_bound()) - np.asarray(pcd.get_min_bound()))
o3d.visualization.draw_geometries([pcd])

In [ ]:
cropped = pcd.crop(mesh_box.get_oriented_bounding_box())
o3d.visualization.draw_geometries([cropped])

# cropped2 = pcd.crop(mesh_box.get_axis_aligned_bounding_box())
# o3d.visualization.draw_geometries([cropped])

In [ ]:
len(np.asarray(cropped.points))

In [ ]:
ureg = UnitRegistry()

In [ ]:
a = (1 * ureg.feet).to('meter') * (1 * ureg.feet).to('meter')
area_per_point = a * 6 / len(np.asarray(cropped.points))
area_per_point

In [ ]:
camera = [1, 0, 0.5]
radius = 4

print("Get all points that are visible from given view point")
_, pt_map = pcd.hidden_point_removal(camera, radius)

print("Visualize result")
pcd_cam = pcd.select_by_index(pt_map)
o3d.visualization.draw_geometries([pcd_cam])

In [ ]:
Pcd = o3d.geometry.PointCloud()

for i in np.linspace(0, 3, 6):
    camera = [1, i, 1]
    
    _, pt_map = pcd.hidden_point_removal(camera, radius)

    pcd_cam = pcd.select_by_index(pt_map)
    
    Pcd += pcd_cam
    
o3d.visualization.draw_geometries([Pcd, mesh_box])

In [ ]:
Pcd

In [ ]:
Pcd = o3d.geometry.PointCloud()

Pcd_set = set()

for i in np.linspace(0, 3, 6):
    camera = [1, i, 1]
    
    _, pt_map = pcd.hidden_point_removal(camera, 3)

    pcd_temp = pcd.select_by_index(pt_map)
    
    pcd_temp_set = set(map(tuple, np.asarray(pcd_temp.points)))
    
    Pcd_set = Pcd_set.union(pcd_temp_set)
    
Pcd_2nparray = np.array(list(Pcd_set))

Pcd.points = o3d.utility.Vector3dVector(Pcd_2nparray)

o3d.visualization.draw_geometries([Pcd])

In [ ]:
Pcd

# Calculate UVC Light Dose

Product Name: ClorDiSys Lantern UV Disinfection System

Irradiance: 150 micro watt per square centimeter at 4 feet  (1.5  watt/m2, r = 1.219 m) <br>
Irradiance: 22  micro watt per square centimeter at 10 feet (0.22 watt/m2, r = 3.048 m)

Since UV Light Intensity is the total amount of energy, it is eqaual to the product between the irradiance (i) and the area of a sphere with its radius equal to the distance (r):

I = i * (4 * pie * r^2)      equation 1

UV Light Intensity (I) is inversely proportional to the square of distance

I = k * (1 / r^2) + b      equation 2

1.5 * (4 * pie * (1.219)^2) = k * (1 / (1.219)^2) + b <br>
0.22 * (4 * pie * (3.048)^2) = k * (1 / (3.048)^2) + b

k = 4.1139, b = 25.2412

In [ ]:
i1 = 1.5 * ureg.watt / (ureg.meter * ureg.meter)
r1 = (4 * ureg.feet).to('meter')

i2 = 0.22 * ureg.watt / (ureg.meter * ureg.meter)
r2 = (10 * ureg.feet).to('meter')

In [ ]:
k = (i1 * (4 * np.pi * (r1)**2) - i2 * (4 * np.pi * (r2)**2)) / (1 / (r1)**2 - 1 / (r2)**2)
b = i1 * (4 * np.pi * (r1)**2) - k * (1 / (r1)**2)

print(k, b)

In [ ]:
Q_ = ureg.Quantity
r = Q_(np.linspace(0.5, 10, 100), 'meter')
Intensity = k * (1 / np.square(r)) + b
irradiance = Intensity / (4 * np.pi * (r**2))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize = (20, 10))

ax[0].plot(r, Intensity)
ax[1].plot(r, irradiance)
ax[1].plot([r1.magnitude, r1.magnitude], [0, 2], '--r', linewidth = 0.5)
ax[1].plot([r2.magnitude, r2.magnitude], [0, 2], '--r', linewidth = 0.5)

ax[1].set_ylim(0, 1.6)

ax[0].set_xlabel("Distance (m)", fontsize=12)
ax[1].set_xlabel("Distance (m)", fontsize=12)
ax[0].set_ylabel("UV Intensity (watt)", fontsize=12)
ax[1].set_ylabel("UV Irradiance (watt/m2)", fontsize=12)

# UVC Irradiance

In [ ]:
camera = [1, 1, 1]
radius = 10

mesh_cylinder = o3d.geometry.TriangleMesh.create_cylinder(radius=0.15, height=0.5)
mesh_cylinder.compute_vertex_normals()
mesh_cylinder.translate(camera)
mesh_cylinder.paint_uniform_color([0.1, 0.9, 0.1])

print("Get all points that are visible from given view point")
_, pt_map = pcd.hidden_point_removal(camera, radius)

print("Visualize result")
pcd_cam = pcd.select_by_index(pt_map)
o3d.visualization.draw_geometries([mesh, pcd_cam, mesh_cylinder])

In [ ]:
Q_ = ureg.Quantity
distance = Q_(np.linalg.norm(np.asarray(pcd_cam.points)-np.array(camera), axis=1), 'feet')

# Point Cloud Color

In [ ]:
pc = o3d.geometry.PointCloud()

point_set = np.array([(x, 0, 0) for x in np.linspace(0, 5, 100)])

pc.points = o3d.utility.Vector3dVector(point_set)

colors = [x.rgb for x in list(red.range_to(blue,100))]
pc.colors = o3d.utility.Vector3dVector(colors)

print(pc)
o3d.visualization.draw_geometries([pc])

In [ ]:
o3d.visualization.draw_geometries([])